Art. 24 Toda água para consumo humano fornecida coletivamente deverá passar por processo de desinfecção ou adição de desinfetante para manutenção dos residuais mínimos, conforme as disposições contidas no Art. 32.

In [1]:
import os
import re
import sys
import pprint
import pandas as pd

In [2]:
from paths import *

In [3]:
# Parameters
cod_ibge = '3548906' # São Carlos
cod_ibge = '3526902' # Limeira
cod_ibge = '3501608' # Americana

In [4]:
# Read Table
df = pd.read_excel(
    os.path.join(output_path, str(cod_ibge), 'dados brutos', 'cadastro', 'cadastro_tratamento_de_agua.xlsx')
)

In [5]:
# Filtra Apenas SAAs
df = df.loc[df['Tipo Da Forma De Abastecimento'] == 'SAA']

# Filtra Apenas Último Ano
df = df[df['Ano De Referência'] == max(df['Ano De Referência'])].copy()
#df.info()
list(df.columns)

['Região Geográfica',
 'Regional De Saúde',
 'Município',
 'Tipo Da Instituição',
 'Sigla Da Instituição',
 'Nome Da Instituição',
 'Cnpj Da Instituição',
 'Nome Do Escritório Regional/Local',
 'Cnpj Do Escritório Regional/Local',
 'Tipo Da Forma De Abastecimento',
 'Código Forma De Abastecimento',
 'Nome Da Forma De Abastecimento',
 'Nome Da Eta/Uta',
 'Ano De Referência',
 'Data De Registro',
 'Data De Preenchimento',
 'Captação Superficial',
 'Captação Subterrânea',
 'Captação De Água De Chuva',
 'Cep',
 'Endereço',
 'Número',
 'Ddd',
 'Telefone',
 'Email',
 'Tempo Médio Diário De Funcionamento',
 'Etapa Pré-Oxidação',
 'Etapa Mistura Rápida/Coagulação',
 'Etapa Floculação',
 'Etapa Decantação',
 'Etapa Flotação',
 'Tipo De Filtração',
 'Número De Filtros',
 'Etapa Desinfecção',
 'Desinfecção Com Cloro  Gás Ou Hipoclorito',
 'Desinfecção Com Cloramina',
 'Desinfecção Com Dióxido Cloro',
 'Desinfecção Com Ozônio',
 'Desinfecção Com Uv',
 'Outro Agente Desinfetante',
 'Rad Cloro Resid

<br>

# Artigo 24

In [6]:
list_iden = [
    'Código Forma De Abastecimento',
    'Tipo Da Forma De Abastecimento',
    'Nome Da Forma De Abastecimento',    
    'Ano De Referência',
    'Etapa Desinfecção',
]

list_desinfeccao = [
    'Desinfecção Com Cloro  Gás Ou Hipoclorito',
    'Desinfecção Com Cloramina',
    'Desinfecção Com Dióxido Cloro',
    'Desinfecção Com Ozônio',
    'Desinfecção Com Uv'
]

# Lista de Colunas de Interesse
list_cols = list_iden + list_desinfeccao

In [28]:
# Seleciona Colunas
df_ = df[list_cols].copy()
df_.reset_index(drop=True, inplace=True)
df_

,Código Forma De Abastecimento,Tipo Da Forma De Abastecimento,Nome Da Forma De Abastecimento,Ano De Referência,Etapa Desinfecção,Desinfecção Com Cloro Gás Ou Hipoclorito,Desinfecção Com Cloramina,Desinfecção Com Dióxido Cloro,Desinfecção Com Ozônio,Desinfecção Com Uv
0,S350160000001,SAA,ETA,2022,S,S,N,N,N,N


In [29]:
for i, row in df_.iterrows():
    desin = df_.loc[i, 'Etapa Desinfecção']
    forma = df_.loc[i, 'Nome Da Forma De Abastecimento']
    tem_desinfeccao = True if desin == 'S' else False
    atendimento = 'atende' if tem_desinfeccao else 'não atende'
    
    # 
    df_temp = pd.DataFrame(df_.loc[i, list_desinfeccao].T.copy())
    display(df_temp)
    df_temp.columns = ['sistemas']
        
    # 
    list_tratamentos = list(df_temp[df_temp['sistemas'] == 'S'].index.values)
    list_tratamentos = [t.lower() for t in list_tratamentos]
    list_tratamentos = [re.sub('\s\s+', ' ', t) for t in list_tratamentos]
    list_tratamentos = ', '.join(list_tratamentos)    

    msg = '''
    De acordo com as informações obtidas no Siságua,
    o município de {municipio} {atende} o artigo 24 da Portaria 888,
    que determia que "toda água para consumo humano
    fornecida coletivamente deverá passar por processo de
    desinfecção ou adição de desinfetante para manutenção dos 
    residuais mínimos, conforme as disposições contidas
    no Art. 32".

    No Siságua é informado que o sistema de tratamento 
    da {forma} realiza a "{tratamento}".
    '''.format(
        municipio=cod_ibge,
        atende=atendimento,
        forma=forma,
        tratamento=list_tratamentos
    )
    
    # 
    pprint.pprint(msg)

,0
Desinfecção Com Cloro Gás Ou Hipoclorito,S
Desinfecção Com Cloramina,N
Desinfecção Com Dióxido Cloro,N
Desinfecção Com Ozônio,N
Desinfecção Com Uv,N


('\n'
 '    De acordo com as informações obtidas no Siságua,\n'
 '    o município de 3501608 atende o artigo 24 da Portaria 888,\n'
 '    que determia que "toda água para consumo humano\n'
 '    fornecida coletivamente deverá passar por processo de\n'
 '    desinfecção ou adição de desinfetante para manutenção dos \n'
 '    residuais mínimos, conforme as disposições contidas\n'
 '    no Art. 32".\n'
 '\n'
 '    No Siságua é informado que o sistema de tratamento \n'
 '    da ETA realiza a "desinfecção com cloro gás ou hipoclorito".\n'
 '    ')


<br>

# Artigo 24, Parágrafo Único

Na tabela não tem informações se é, ou não, superficial.

31.03.2022

In [30]:
list_filtracao = [
    'Tipo De Filtração',
    'Número De Filtros',
]

# Lista de Colunas de Interesse
list_cols = list_iden + list_filtracao

In [31]:
# Seleciona Colunas
df_ = df[list_cols].copy()
df_.reset_index(drop=True, inplace=True)
df_

,Código Forma De Abastecimento,Tipo Da Forma De Abastecimento,Nome Da Forma De Abastecimento,Ano De Referência,Etapa Desinfecção,Tipo De Filtração,Número De Filtros
0,S350160000001,SAA,ETA,2022,S,FILTRAÇÃO RÁPIDA,6.0


In [32]:
for i, row in df_.iterrows():
    filt = df_.loc[i, 'Tipo De Filtração']
    forma = df_.loc[i, 'Nome Da Forma De Abastecimento']
    tem_filtracao = True if pd.isna(filt) else False
    atendimento = 'atende' if tem_filtracao else 'não atende'
    
    # 
    #df_temp = pd.DataFrame(df_.loc[i, list_desinfeccao].T.copy())
    #df_temp.columns = ['sistemas']
        
    # 
    #list_tratamentos = list(df_temp[df_temp['sistemas'] == 'S'].index.values)
    #list_tratamentos = [t.lower() for t in list_tratamentos]
    #list_tratamentos = [re.sub('\s\s+', ' ', t) for t in list_tratamentos]
    #list_tratamentos = ', '.join(list_tratamentos)    

    msg = '''
    De acordo com as informações obtidas no Siságua,
    o município de {municipio} {atende} o artigo 24 da Portaria 888,
    que determia que "toda água para consumo humano
    fornecida coletivamente deverá passar por processo de
    desinfecção ou adição de desinfetante para manutenção dos 
    residuais mínimos, conforme as disposições contidas
    no Art. 32".

    No Siságua é informado que o sistema de tratamento 
    da {forma} realiza a "{tratamento}".
    '''.format(
        municipio=cod_ibge,
        atende=atendimento,
        forma=forma,
        tratamento=list_tratamentos
    )
    
    # 
    pprint.pprint(msg)

('\n'
 '    De acordo com as informações obtidas no Siságua,\n'
 '    o município de 3501608 não atende o artigo 24 da Portaria 888,\n'
 '    que determia que "toda água para consumo humano\n'
 '    fornecida coletivamente deverá passar por processo de\n'
 '    desinfecção ou adição de desinfetante para manutenção dos \n'
 '    residuais mínimos, conforme as disposições contidas\n'
 '    no Art. 32".\n'
 '\n'
 '    No Siságua é informado que o sistema de tratamento \n'
 '    da ETA realiza a "desinfecção com cloro gás ou hipoclorito".\n'
 '    ')
